In [4]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import sqlite3

BASE_DIR = Path.cwd().parent
db_path = BASE_DIR / 'data' / "chinook.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor() 
query = """SELECT
    i.invoiceId,
    i.invoiceDate,
    i.total AS invoiceTotal,
    ii.invoiceLineId,
    ii.unitPrice AS itemUnitPrice,

    c.customerId,
    c.firstName AS customerFirstName,
    c.lastName AS customerLastName,
    c.company AS customerCompany,
    c.country AS customerCountry,

    e.employeeId,
    e.firstName AS employeeFirstName,
    e.lastName AS employeeLastName,
    e.title AS employeeTitle,
    e.ReportsTo AS employeeReportsTo,

    t.trackId,
    t.name AS trackName,
    t.composer,
    t.Milliseconds,

    al.title AS albumTitle,
    ar.name AS artistName,

    g.name AS genreName,
    mt.name AS mediaTypeName
   

FROM
    invoice_items AS ii
LEFT JOIN invoices AS i ON ii.invoiceId = i.invoiceId
LEFT JOIN customers AS c ON i.customerId = c.customerId
LEFT JOIN employees AS e ON c.supportRepId = e.employeeId
LEFT JOIN tracks AS t ON ii.trackId = t.trackId
LEFT JOIN albums AS al ON t.albumId = al.albumId
LEFT JOIN artists AS ar ON al.artistId = ar.artistId
LEFT JOIN genres AS g ON t.genreId = g.genreId
LEFT JOIN media_types AS mt ON t.mediaTypeId = mt.mediaTypeId;
"""
df = pd.read_sql_query(query, conn)
conn.close()  
df.head()

,InvoiceId,InvoiceDate,invoiceTotal,InvoiceLineId,itemUnitPrice,CustomerId,customerFirstName,customerLastName,customerCompany,customerCountry,...,employeeTitle,employeeReportsTo,TrackId,trackName,Composer,Milliseconds,albumTitle,artistName,genreName,mediaTypeName
0,1,2009-01-01 00:00:00,1.98,1,0.99,2,Leonie,Köhler,None,Germany,...,Sales Support Agent,2,2,Balls to the Wall,None,342562,Balls to the Wall,Accept,Rock,Protected AAC audio file
1,1,2009-01-01 00:00:00,1.98,2,0.99,2,Leonie,Köhler,None,Germany,...,Sales Support Agent,2,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,Restless and Wild,Accept,Rock,Protected AAC audio file
2,2,2009-01-02 00:00:00,3.96,3,0.99,4,Bjørn,Hansen,None,Norway,...,Sales Support Agent,2,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file
3,2,2009-01-02 00:00:00,3.96,4,0.99,4,Bjørn,Hansen,None,Norway,...,Sales Support Agent,2,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file
4,2,2009-01-02 00:00:00,3.96,5,0.99,4,Bjørn,Hansen,None,Norway,...,Sales Support Agent,2,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file


In [ ]:
# df.info()
# df.describe(include=['object'])
# df.describe()

# df.head(5)
# df.head()
# df.describe()
# df.shape
# df.dtypes
# df.duplicated().sum()
df.isnull().mean() * 100
# df.describe(include='all')

InvoiceId             0.000000
InvoiceDate           0.000000
invoiceTotal          0.000000
InvoiceLineId         0.000000
itemUnitPrice         0.000000
CustomerId            0.000000
customerFirstName     0.000000
customerLastName      0.000000
customerCompany      83.035714
customerCountry       0.000000
EmployeeId            0.000000
employeeFirstName     0.000000
employeeLastName      0.000000
employeeTitle         0.000000
employeeReportsTo     0.000000
TrackId               0.000000
trackName             0.000000
Composer             26.607143
Milliseconds          0.000000
albumTitle            0.000000
artistName            0.000000
genreName             0.000000
mediaTypeName         0.000000
dtype: float64

In [4]:
print('Total de receita por ano')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
receita_anual = (
    df.groupby(df['InvoiceDate'].dt.year)['invoiceTotal']
      .sum()
      .reset_index(name='receita')
      .rename(columns={'InvoiceDate': 'ano'})
)
receita_anual.head(10).sort_values(by='receita', ascending=False)

Total de receita por ano


,ano,receita
1,2010,4329.45
3,2012,4256.25
2,2011,4126.40
4,2013,4125.04
0,2009,4011.48


In [5]:
print('Quais são os 10 compositores (Composer) mais vendidos em termos de quantidade de faixas (quantity)?')

top10_compositores = df.groupby('genreName')['InvoiceLineId'].count().nlargest(10).reset_index(name='quantidade_de_faixas').rename(columns={'InvoiceLineId': 'Quantidade de Faixas'})
top10_compositores

Quais são os 10 compositores (Composer) mais vendidos em termos de quantidade de faixas (quantity)?


,genreName,quantidade_de_faixas
0,Rock,835
1,Latin,386
2,Metal,264
3,Alternative & Punk,244
4,Jazz,80
5,Blues,61
6,TV Shows,47
7,Classical,41
8,R&B/Soul,41
9,Reggae,30


In [36]:
print('Qual é o valor médio gasto por cliente (invoiceTotal) nos 5 países com o maior número de clientes?')
total_clientes = df['CustomerId'].nunique()
total_clientes = df.groupby('customerCountry')['CustomerId'].nunique().reset_index(name='total_clientes')
# total_clientes.sort_values(by='total_clientes', ascending=False)
top5_clientes_por_pais = total_clientes.nlargest(5, 'total_clientes')[['customerCountry', 'total_clientes']].reset_index()
# top5_clientes_por_pais

media_gasto_cliente = (
				df.groupby('customerCountry')['invoiceTotal'].mean().reset_index(name='media_gasto_cliente')
			)
media_gasto_cliente = media_gasto_cliente.merge(top5_clientes_por_pais, on='customerCountry', how='inner')
media_gasto_cliente = media_gasto_cliente[['customerCountry', 'media_gasto_cliente',	'total_clientes']]
media_gasto_cliente.sort_values(by='media_gasto_cliente', ascending=False)

Qual é o valor médio gasto por cliente (invoiceTotal) nos 5 países com o maior número de clientes?


,customerCountry,media_gasto_cliente,total_clientes
4,USA,9.447490,13
3,Germany,9.161053,4
2,France,9.063684,5
1,Canada,8.848553,8
0,Brazil,8.826842,5


In [57]:
tipos_midia = df['mediaTypeName'].unique()
total_vendido_por_tipo = df.groupby('mediaTypeName')['invoiceTotal'].sum()
porcentagem_vendido_por_tipo = round((total_vendido_por_tipo / total_vendido_por_tipo.sum()) * 100, 4)
# porcentagem_vendido_por_tipo
vendas_por_tipo_midia = pd.DataFrame({'total_vendido': total_vendido_por_tipo, 'porcentagem_vendido': porcentagem_vendido_por_tipo}).reset_index()
vendas_por_tipo_midia = vendas_por_tipo_midia.sort_values(by='total_vendido', ascending=False)
vendas_por_tipo_midia

,mediaTypeName,total_vendido,porcentagem_vendido
1,MPEG audio file,17838.27,85.5609
3,Protected MPEG-4 video file,1775.90,8.5181
2,Protected AAC audio file,1160.11,5.5644
0,AAC audio file,48.58,0.2330
4,Purchased AAC audio file,25.76,0.1236


In [72]:
dates = df['InvoiceDate'].dt.day_name()
# dates
vendas_por_dia = df.groupby(dates)['InvoiceId'].nunique().reset_index().sort_values(by='InvoiceId', ascending=False)
vendas_por_dia = vendas_por_dia.rename(columns={'InvoiceId': 'total_vendas', 'InvoiceDate': 'dia_da_semana'})
vendas_por_dia

,dia_da_semana,total_vendas
3,Sunday,60
1,Monday,59
0,Friday,59
6,Wednesday,59
4,Thursday,59
2,Saturday,58
5,Tuesday,58


In [ ]:
total_clientes = df['CustomerId'].nunique()
total_clientes_compra_rock = df[df['genreName'] == 'Rock']['CustomerId'].nunique()
total_clientes_compra_jazz = df[df['genreName'] == 'Jazz']['CustomerId'].nunique()
porcentagem = round(total_clientes_compra_jazz / total_clientes_compra_rock * 100, 2) 
porcentagem

 Dos clientes que compraram músicas do gênero "Rock", qual a porcentagem deles que também já comprou músicas do gênero "Jazz"?


54.24

In [ ]:
media_venda_por_vendedor = df.groupby('employeeFirstName')['invoiceTotal'].mean().reset_index(name='media_venda').sort_values(by='media_venda', ascending=False)
round(media_venda_por_vendedor,2)


,employeeFirstName,media_venda
2,Steve,9.49
0,Jane,9.33
1,Margaret,9.12


In [ ]:
generos_artistas = df[['artistName', 'genreName']].drop_duplicates()
generos_artistas = generos_artistas.groupby('artistName').agg(lista_de_generos=('genreName', list), num_generos=('genreName', 'nunique')).reset_index()
artistas_diversidade = generos_artistas.sort_values(by='num_generos', ascending=False).head(5)
artistas_diversidade

Quais artistas têm álbuns em mais de um gênero musical? Liste os 5 artistas com a maior diversidade de gêneros


,artistName,lista_de_generos,num_generos
69,Iron Maiden,"[Rock, Metal, Heavy Metal, Blues]",4
14,Audioslave,"[Rock, Alternative & Punk, Alternative]",3
72,Jamiroquai,"[Rock, R&B/Soul, Electronica/Dance]",3
59,Gilberto Gil,"[Soundtrack, Latin, Jazz]",3
16,Battlestar Galactica,"[TV Shows, Science Fiction, Sci Fi & Fantasy]",3


In [7]:
correlacao_duracao_venda = df[['Milliseconds', 'itemUnitPrice']].corr().drop_duplicates()
correlacao_duracao_venda 

# print(df[['Milliseconds', 'itemUnitPrice']].describe())

,Milliseconds,itemUnitPrice
Milliseconds,1.000000,0.933533
itemUnitPrice,0.933533,1.000000
